In [1]:
import requests
import random
from collections import Counter

In C:\Users\hyoun\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\hyoun\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\hyoun\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\hyoun\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\hyoun\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

In [2]:
# iris 데이터 다운로드
iris_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
r = requests.get(iris_url)
open('iris.data', 'wb').write(r.content)
# 데이터 분리
vectors = []
answers = []
with open('iris.data', 'r') as f:
    for line in f:
        if len(line.strip()) != 0:
            items = line.strip().split(",")
            vectors.append([float(i) for i in items[:4]])
            answers.append(items[4])
            
zipped = list(zip(vectors, answers))
random.shuffle(zipped)
train_size = int(len(vectors) * 0.8)
train_x, train_y = zip(*zipped[:train_size])
test_x, test_y = zip(*zipped[train_size:])

In [3]:
# gini값 구하는 함수 
def gini_score(items):
    counter = Counter(items)
    total_count = sum(counter.values())
    return 1 - sum((c / total_count) ** 2 for c in counter.values())

In [4]:
# Decision Tree에서 분할점 찾는 함수 지정한 컬럼에서 Gini값과 값을 알 수 있음. 
def find_split_point_of_a_field(pairs):
    S = sorted(pairs, key=lambda x: x[0])
    gini_min = 99
    split_point = 0
    for i in range(1, len(S)):
        if S[i-1][0] != S[i][0]:
            prop = i/len(S)
            gini = gini_score([s[1] for s in S[:i]]) * prop + gini_score([s[1] for s in S[i:]]) * (1 - prop)
            if gini < gini_min:
                gini_min = gini
                split_point = (S[i-1][0] + S[i][0])/2

    return split_point, gini_min

In [23]:
print("첫번째 Columns : {}".format(find_split_point_of_a_field(zip([item[0] for item in train_x], train_y))))
print("두번째 Columns : {}".format(find_split_point_of_a_field(zip([item[1] for item in train_x], train_y))))
print("세번째 Columns : {}".format(find_split_point_of_a_field(zip([item[2] for item in train_x], train_y))))
print("네번째 Columns : {}".format(find_split_point_of_a_field(zip([item[3] for item in train_x], train_y))))

첫번째 Columns : (5.45, 0.4460317460317461)
두번째 Columns : (3.3499999999999996, 0.5104640151515152)
세번째 Columns : (2.5999999999999996, 0.3278481012658228)
네번째 Columns : (0.8, 0.3278481012658228)


In [5]:
# 각각 columns에 대해서 gini계수 계산하고 분할점으로 선택할 column 및 값 선택해서 분해하는 부분 
def split_data(X, Y):
    num_fields = len(X[0])

    gini_min, sp_min, fid_min = 99, -1, -1
    for fid in range(num_fields):
        sp, gini = find_split_point_of_a_field(zip([item[fid] for item in X], Y))
        if gini < gini_min:
            gini_min, sp_min, fid_min = gini, sp, fid
    ret = {}
    ret["sp"], ret["gini"], ret["fid"] = sp_min, gini_min, fid_min
    ret["left"] = tuple(zip(*[(x, y) for x, y in zip(X, Y) if x[fid_min] < sp_min]))
    ret["right"] = tuple(zip(*[(x, y) for x, y in zip(X, Y) if x[fid_min] >= sp_min]))
    return ret

In [15]:
split_data(train_x,train_y)

{'sp': 2.5999999999999996,
 'gini': 0.3278481012658228,
 'fid': 2,
 'left': (([4.8, 3.4, 1.9, 0.2],
   [5.2, 4.1, 1.5, 0.1],
   [5.2, 3.5, 1.5, 0.2],
   [5.4, 3.7, 1.5, 0.2],
   [4.7, 3.2, 1.6, 0.2],
   [5.4, 3.4, 1.5, 0.4],
   [5.7, 3.8, 1.7, 0.3],
   [4.8, 3.1, 1.6, 0.2],
   [4.4, 3.0, 1.3, 0.2],
   [5.1, 3.8, 1.6, 0.2],
   [5.1, 3.5, 1.4, 0.2],
   [5.1, 3.7, 1.5, 0.4],
   [5.3, 3.7, 1.5, 0.2],
   [5.7, 4.4, 1.5, 0.4],
   [5.0, 3.5, 1.3, 0.3],
   [5.2, 3.4, 1.4, 0.2],
   [5.8, 4.0, 1.2, 0.2],
   [5.0, 3.2, 1.2, 0.2],
   [4.6, 3.1, 1.5, 0.2],
   [5.0, 3.3, 1.4, 0.2],
   [5.4, 3.9, 1.7, 0.4],
   [5.4, 3.4, 1.7, 0.2],
   [5.5, 3.5, 1.3, 0.2],
   [4.6, 3.4, 1.4, 0.3],
   [4.6, 3.2, 1.4, 0.2],
   [5.0, 3.6, 1.4, 0.2],
   [5.0, 3.4, 1.5, 0.2],
   [5.0, 3.5, 1.6, 0.6],
   [5.1, 3.5, 1.4, 0.3],
   [5.1, 3.8, 1.5, 0.3],
   [5.0, 3.0, 1.6, 0.2],
   [4.6, 3.6, 1.0, 0.2],
   [4.7, 3.2, 1.3, 0.2],
   [5.1, 3.8, 1.9, 0.4],
   [4.3, 3.0, 1.1, 0.1],
   [5.1, 3.4, 1.5, 0.2],
   [4.8, 3.4, 1.6, 0.2],


In [24]:
# 분할된 것을 계속 받아와 분할하도록 작업 수행 (재귀적 수행하도록)

def decision_tree(X, Y, threshold):
    original_gini = gini_score(Y)
    node = split_data(X, Y)
    counter = Counter(Y)
    if original_gini < node['gini'] + threshold:
        ans, cnt = counter.most_common(1)[0]
        return ans, cnt / sum(counter.values())
    else:
        # 새롭게 Tree 분할하는 재귀함수 위치 
        node['left'] = decision_tree(node['left'][0], node['left'][1], threshold)
        node['right'] = decision_tree(node['right'][0], node['right'][1], threshold)
        return node

In [26]:
decision_tree(train_x,train_y,0)

{'sp': 2.5999999999999996,
 'gini': 0.3278481012658228,
 'fid': 2,
 'left': {'sp': 4.35,
  'gini': 0.0,
  'fid': 0,
  'left': ('Iris-setosa', 1.0),
  'right': {'sp': 4.45,
   'gini': 0.0,
   'fid': 0,
   'left': ('Iris-setosa', 1.0),
   'right': {'sp': 4.55,
    'gini': 0.0,
    'fid': 0,
    'left': ('Iris-setosa', 1.0),
    'right': {'sp': 4.65,
     'gini': 0.0,
     'fid': 0,
     'left': {'sp': 3.1500000000000004,
      'gini': 0.0,
      'fid': 1,
      'left': ('Iris-setosa', 1.0),
      'right': {'sp': 3.3,
       'gini': 0.0,
       'fid': 1,
       'left': ('Iris-setosa', 1.0),
       'right': {'sp': 3.5,
        'gini': 0.0,
        'fid': 1,
        'left': ('Iris-setosa', 1.0),
        'right': ('Iris-setosa', 1.0)}}},
     'right': {'sp': 4.75,
      'gini': 0.0,
      'fid': 0,
      'left': {'sp': 1.4500000000000002,
       'gini': 0.0,
       'fid': 2,
       'left': ('Iris-setosa', 1.0),
       'right': ('Iris-setosa', 1.0)},
      'right': {'sp': 4.85,
       'gini':

In [7]:
def predict(x, tree):
    if 'fid' not in tree:
        return tree
    if x[tree['fid']] < tree['sp']:
        return predict(x, tree['left'])
    else:
        return predict(x, tree['right'])

In [8]:
tree = decision_tree(train_x, train_y, 0)
success = 0
for x, y in zip(test_x, test_y):
    p = predict(x, tree)
    if p[0] == y:
        success += 1
print("accuracy: ", success / len(test_x))

accuracy:  0.9333333333333333


In [28]:
tree = decision_tree(train_x, train_y, 0.1)
success = 0
for x, y in zip(test_x, test_y):
    p = predict(x, tree)
    if p[0] == y:
        success += 1
print("accuracy: ", success / len(test_x))

accuracy:  0.9333333333333333


In [34]:
tree = decision_tree(train_x, train_y, 0.32)
success = 0
for x, y in zip(test_x, test_y):
    p = predict(x, tree)
    if p[0] == y:
        success += 1
print("accuracy: ", success / len(test_x))

accuracy:  0.9333333333333333


In [35]:
decision_tree(train_x, train_y, 0.32)

{'sp': 2.5999999999999996,
 'gini': 0.3278481012658228,
 'fid': 2,
 'left': ('Iris-setosa', 1.0),
 'right': {'sp': 4.75,
  'gini': 0.11683627037312819,
  'fid': 2,
  'left': ('Iris-versicolor', 0.9705882352941176),
  'right': ('Iris-virginica', 0.9111111111111111)}}